* Look through keyword data to figure out its temporal and user dimensions
* Get the users in keyword dataset who replied to others in keyword search (on topic replies)
* Get those who were replied to in the keyword data set

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

Basic checks for the keyword data set

In [3]:
tp=pd.read_csv('/home/mako0970/ReplyPaperNew/data/Sandy_search_tweets.csv',sep=';',\
iterator=True, chunksize=1000,usecols=['id','username','user_id','date','location','source','replyWhoID'])
data=pd.concat(list(tp), ignore_index=True)

In [4]:
data['date_obj']=pd.to_datetime(data.date)

In [5]:
len(data)

22150275

In [6]:
len(data.user_id.unique())

8000942

In [7]:
min(data.date_obj)

Timestamp('2012-10-24 23:58:06')

In [28]:
max(data.date_obj)

Timestamp('2013-04-05 23:56:57')

Limit the dataset to until 11/30/2012 - consistent with RT paper

In [7]:
temp=data[data.date_obj<=pd.to_datetime('2012-11-30 23:59:59')]

In [8]:
len(temp)

16233401

In [9]:
len(temp.user_id.unique())

5863168

Look at the tweets that have replies and unique replied to users

In [10]:
len(temp.replyWhoID[temp.replyWhoID!=0].unique())

630514

Check how many of the replied to users are already in the keyword dataset

In [11]:
len(set(temp.replyWhoID[temp.replyWhoID!=0].unique()) & set(temp.user_id.unique()))

409699

Get the list of users to whom our users replied, and who are also themselves in the Keyword search data set

In [12]:
repliedInKeyword=pd.DataFrame(list(set(temp.replyWhoID[temp.replyWhoID!=0].unique()) & set(temp.user_id.unique())))

In [57]:
repliedInKeyword.to_csv("RepliedToUsersAlsoInKeywordSet.csv", header=False, index=False)

also get usernames for the bb file (doesn't have the replyWhoID yet)

In [57]:
repKeyUsername=(temp.username[temp.replyWhoID.isin(repliedInKeyword[0].tolist())]).unique()

In [66]:
np.savetxt('/home/mako0970/ReplyPaperNew/data/RepliedToUsersAlsoInKeywordSetUsername.csv', repKeyUsername, fmt='%s',delimiter=',')

Get the list of users who replied to the tweets of those who are also in the Keyword search data set

In [14]:
usersWithRepliesInKeyword=temp.user_id[temp.replyWhoID.isin(repliedInKeyword[0].tolist())].unique()

In [15]:
len(usersWithRepliesInKeyword)

552086

Get the final list of users in Keyword set: both sides of the reply process

In [21]:
bothSides=list(set(usersWithRepliesInKeyword) | set(repliedInKeyword[0].tolist()))

In [22]:
len(bothSides)

806151

In [ ]:
(np.asarray(bothSides)).tofile('/home/mako0970/ReplyPaperNew/data/keywordReplyBothSides.csv', sep=',')

In [27]:
(np.asarray(usersWithRepliesInKeyword)).tofile('/home/mako0970/ReplyPaperNew/data/keywordRepliersOnly.csv', sep=',')